### In this project we are making character based model which predicts the next character based on previous text

We are using tine shakespere dataset where we have all the poems written by shakespere in text document format


In [41]:
## Downloading tiny shakespere dataset
!wget https://raw.githubusercontent.com/Karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-04-04 19:07:46--  https://raw.githubusercontent.com/Karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-04-04 19:07:46 (17.3 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [42]:
## reading the text file
with open('input.txt','r',encoding='utf-8') as f:
  text=f.read()

In [43]:
print("length of dataset in characters: ",len(text))

length of dataset in characters:  1115394


In [44]:
## first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [45]:
## here are all the unique characters that occur in the text

## text -> sequece of char , set -> find unique char, list -> make list
chars=sorted(list(set(text)))

vocab_size=len(chars)

print("Unique character in this text are : ",''.join(chars))
print(vocab_size)

Unique character in this text are :  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## create a mapping from character to integer

In [46]:
## stoi is dictnary having elements as character:index
stoi = {ch:i for i,ch in enumerate(chars)}

## itos is dictnary having elements as index:character
itos = {i:ch for i,ch in enumerate(chars)}

## function to convert every character in string to it desired index acc to stoi and returns list
encode = lambda s: [stoi[i] for i in s]

## function to convert every index in list to it desired character and return a string
decode = lambda s: ''.join([itos[index] for index in s])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


## Encode the entire text of shakespeare poem and store it into torch.tensor

In [47]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print("length of data (encoded text file) :",len(data))
print("Shape of data (encoded text file) : ",data.shape)
print("dtatype of data (encoded text file) :",data.dtype)
print(data[:1000])

length of data (encoded text file) : 1115394
Shape of data (encoded text file) :  torch.Size([1115394])
dtatype of data (encoded text file) : torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0

## split the data into traning and testing

Note : we can't use train test split library because we have sequential data where sequence matters

In [48]:
## taking first 90% data for training and rest data for testing
n=int(0.9*len(data))
train_data=data[:n]
test_data=data[n:]

In [49]:
## defining block size to send to model for training
block_size=8

## defining sample training and testing data with their context and target integer
x=train_data[:block_size]
y=train_data[1:block_size+1]

print("sample training data ",x)
print("sample testing data ",y)

## printing how the dataset will look like
for t in range(block_size):
  context= x[:t+1]
  target= y[t]
  print("when context is ",context," the target is ",target)

sample training data  tensor([18, 47, 56, 57, 58,  1, 15, 47])
sample testing data  tensor([47, 56, 57, 58,  1, 15, 47, 58])
when context is  tensor([18])  the target is  tensor(47)
when context is  tensor([18, 47])  the target is  tensor(56)
when context is  tensor([18, 47, 56])  the target is  tensor(57)
when context is  tensor([18, 47, 56, 57])  the target is  tensor(58)
when context is  tensor([18, 47, 56, 57, 58])  the target is  tensor(1)
when context is  tensor([18, 47, 56, 57, 58,  1])  the target is  tensor(15)
when context is  tensor([18, 47, 56, 57, 58,  1, 15])  the target is  tensor(47)
when context is  tensor([18, 47, 56, 57, 58,  1, 15, 47])  the target is  tensor(58)


## Macking batches of input data

In [50]:
## setting seed so that random number so generated is same
torch.manual_seed(1337)
batch_size=4 ## we try to send 4 batches to model for training
block_size=8 ## define the context length

## function to genrate a small batch of data of inputs x and target y
def get_batch(split):
  data = train_data if split=="train" else test_data
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print("-------")

for b in range(batch_size):
  for t in range(block_size):
    context=xb[b,:t+1]    ## since in xb is tensor so if would be list it would be similar to xb[b[:t+1]]
    target=yb[b,t]
    print("when input is ",context," the target is ",target)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is  tensor([24])  the target is  tensor(43)
when input is  tensor([24, 43])  the target is  tensor(58)
when input is  tensor([24, 43, 58])  the target is  tensor(5)
when input is  tensor([24, 43, 58,  5])  the target is  tensor(57)
when input is  tensor([24, 43, 58,  5, 57])  the target is  tensor(1)
when input is  tensor([24, 43, 58,  5, 57,  1])  the target is  tensor(46)
when input is  tensor([24, 43, 58,  5, 57,  1, 46])  the target is  tensor(43)
when input is  tensor([24, 43, 58,  5, 57,  1, 46, 43])  the target is  tensor(39)
when input is  tensor([44])  the target is  tensor(

## Using bigram language model

In [51]:
!pip install torch

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    ## idx and target are both (B,T) tensor of integrs
    logists=self.token_embedding_table(idx) ## (B,T,C)

    if targets is None:
      loss=None
    else:
      B,T,C =logists.shape
      logists=logists.view(B*T,C)
      targets=targets.view(-1)
      loss=F.cross_entropy(logists,targets)

    return logists, loss

  def generate(self,idx,max_new_token):
    ## idx is (B,T) array of indices in the current context
    for _ in range(max_new_token):
      ## get the predictions
      logists, loss=self(idx)
      ## focus only on the last time step
      logists = logists[:,-1,:] ## becomes (B,C)
      probs=F.softmax(logists, dim=1) #(B,C)
      # sample from the distribution
      idx_next=torch.multinomial(probs,num_samples=1) # (B,1)
      ## append sampled index to the running sequence
      idx=torch.cat((idx,idx_next),dim=1) # (B,T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logists,loss= m.forward(xb,yb)
print(logists.shape)
print(loss)

idx=torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


### Brief explaination about terms B,T,C
B -> batch size (here 32)

T -> time step or block size (here 8)

C -> channel or vocab size or size of embedding vector (here 65)

Note : here in forward fucntion of BigramModel the logits and target are flattend because we are using cross validation frunction for calculating loss so this cross validation function expect :

1. logits (input) as a 2d vector (N,C) where N->tokens and C-> no. of classes(here vocab size)
so we flatten logits from shape (B,T,C) to (B*T,C)

2. target as 1d vector (N) were N->token
so we flatten target from shape (B,T) to (B*T)

here B*T represent all the token in single line since B(Batch_Size) * T(block_size)

In [53]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [54]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [55]:
## Using AdmaW optimizer in pytorch here m.parameter()-> passes all the trainable parameter to optimiser AND lr -> learning rate
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
## note : the more we run this codeblock more the loss will be minimised

## minimising the loss
batch_size=32

## range is similar to epoch in tensorflow
for steps in range(1000):

  # sample a batch of data
  xb,yb=get_batch('train')

  #evaluate the loss by passing x_train and y_train to model
  logists,loss=m(xb,yb)
  # zero out the gradient
  optimizer.zero_grad(set_to_none=True)
  # compute gradient
  loss.backward()
  # update model parameter
  optimizer.step()

print(loss.item())

3.7218432426452637


In [57]:
## now after minimising the loss if again try model to print next character for the character embedded as zero -> it gives better result
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-


Note  : print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

this line means :
 - Starting with a blank/zero token
 - Asking the model "what character should come next?" 100 times
 - Converting the resulting sequence of token IDs back to readable text

### **Note**: till here we have build a very simple model which predicts what comes next based on the last character of context being provided to model

## Self attention blocks for procecessing tokens


In [58]:
## consider the following toy example

## setting seed to get same result everytime
torch.manual_seed(1337)
B,T,C = 4,8,2 ## batch, time_step, channel
x=torch.randn(B,T,C) ## random initialisation of tensor x with shape(B,T,C)
x.shape

torch.Size([4, 8, 2])

## note : here the concept we are using to getnext character is

for predicting the next character we just take average of all the charater appearing before the current character to predict the next character

## Method 1 : Using appropriate for loop for getting average of each channel number with it above numbers

In [59]:
## initialising a tensor named xbow filled dwith zeros of shape (B,T,C)
xbow=torch.zeros((B,T,C))

for b in range(B):
  for t in range(T):
    ## collecting the previous token from current token
    xprev=x[b,:t+1] ## (t,C)
    ## adding the average till the current term in xbow
    xbow[b,t]=torch.mean(xprev,0)

In [60]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [61]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

here we can see 1st row remains same but 2nd row is average of 1st and 2nd row
similaty 3rd row is of 1sr,2nd,3rd

note: since B=4, T=8 and C=2 so average is taken among its own channel

example in 3rd row [a b]

where a = 0.1808 + (-0.0894) + 0.1490 / 3

and b= (-0.0700) + (-0.4926) + (-0.3199) /3

## by using
    ## collecting the previous token from current token
    xprev=x[b,:t+1] ## (t,C)
    ## adding the average till the current term in xbow
    xbow[b,t]=torch.mean(xprev,0)

we get xbow which has average of each channel number with it above numbers



## Method 2 : Using matrix multiplication to get average of each channel number with it above numbers

In [62]:
## code for simple matrix multiplication in pytorch
torch.manual_seed(42)

# a=torch.ones(3,3)  ## makea matrix 3X3 of all elemnt zero
a = torch.tril(torch.ones(3,3)) ## makes lower traingular matrix
b = torch.randint(0,10,(3,2)).float()  ## generare (3X2) matrix of random number 0 - 10 as type float number

c = a @ b  ## matrix multiplication of tensor object

print('a=')
print(a)
print('------')
print('b=')
print(b)
print('------')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
------
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [63]:
### now we are aiming to get average of each channel number in b with it above numbers and showing output in c
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) ## makes lower traingular matrix

a= a/torch.sum(a,1,keepdim=True) ## normalise the row of a

b = torch.randint(0,10,(3,2)).float()  ## generare (3X2) matrix of random number 0 - 10 as type float number

c = a @ b  ## matrix multiplication of tensor object

print('a=')
print(a)
print('------')
print('b=')
print(b)
print('------')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


so in this way by normalising the rows of 'a' we can get average of each channel number in b with it above numbers and showing output in c

below is the way where we generate weights (used while training the model) and multiply it with x (randomly generated tensor of shape (B,T,C) )

In [64]:
## weights (lower triangular matrix)
weights=torch.tril(torch.ones(T,T))
## normalise the matrix
weights=weights/torch.sum(weights,1,keepdim=True)

## matrix multiplication in pytorch
xbow2=weights @ x ## here dim of weights = (T,T) but dim of x = (B,T,C)
## so since weight is tensor object so it will automatically adjust its dim to (B,T,C)

xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

## note xbow and xbow2 are same, they are only generated in different ways

In [65]:
## they are same
print(xbow[0])
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


## Method 3 : Using softmax function

In [66]:
## make lower triangular matrix
tril=torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [67]:
## make initial weights as zeros
weight=torch.zeros((T,T))
weight
## it shows it interaction strength or affinity ie, how much from the past we want to aggregate and average up

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [68]:
## it will make zeros in lower triangular matrix to -infinity and 1s to zeros
weight=weight.masked_fill(tril==0,float('-inf'))
weight
## this line show tokens from the past can't communicate by setting them to -infinity we are saying that we will not aggregate anything from those token

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [69]:
## using softmax function : softmax function act a normalisation which make sum of tokens in a row as 1
weight=F.softmax(weight,dim=-1)
weight

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [70]:
xbow3=weight @ x
xbow3

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

## note: xbow, xbow2 and xbow3 are all same only they are generated using different methods

In [71]:
print("first batch of xbow")
print(xbow[0])
print("first batch of xbow2")
print(xbow2[0])
print("first batch of xbow3")
print(xbow3[0])

first batch of xbow
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
first batch of xbow2
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
first batch of xbow3
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


## Method 4 : Self Attention

In [72]:
## earlier approach

## make lower triangular matrix
tril=torch.tril(torch.ones(T,T))
## make initial weights as zeros
weight=torch.zeros((T,T))
## it shows it interaction strength or affinity ie, how much from the past we want to aggregate and average up
## it will make zeros in lower triangular matrix to -infinity and 1s to zeros
weight=weight.masked_fill(tril==0,float('-inf'))
## this line show tokens from the past can't communicate by setting them to -infinity we are saying that we will not aggregate anything from those token
weight=F.softmax(weight,dim=-1)


## till here we were just taking initial weights as zero which indicate there is no relation between the currently predicted value and previous value that's why we get uniform matrix after applying softmax function

## BUT WE KNOW IN SEQUENTIAL DATA THE NEXT PREDICTED VALUE DEPENDS ON PREVIOUS VALUE ALSO DIFFERNT WORDS IN PAST CORRELATE DIFFERENTLY WITH THE CURRENT WORD

## SO WE NEED NEXT DATA SHOULD BE DEPENDENT ON CURRENT DATA THIS PROBLEM IS SOLVED BY SELF ATTENTION

## in self attention each token of a sequence have two vector "query vector" and "key vector" where query vector tell what i am loking for and key vector say what do i contain
## to find affifnity between the tokens or we can s which all token align with current token we find dot product of current query vector will all of the key vector of token present in sequence, the key vector getting very high amount in dot product align more with that query vector

In [73]:
torch.manual_seed(1337)
B,T,C=4,8,32 ## batch,total token in sequence,channel (embedding dimension)
x=torch.randn(B,T,C)

In [74]:
### head of self attention
head_size=16 ## a hyperparameter

## key and query are used to convert are the input x (B,T,C) to output y (B,T,head_size)
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bais=False)

## making k(key) and q(query) for each input x
k=key(x)
q=query(x)

## calculating weight
weight= q @ k.transpose(-2,-1) ## (B,T,16) @ (B,16,T) ---> (B,T,T)
## so for every input sequence we will have T X T matrix which represent the affinities

## using self attenstion to genrate weights
tril=torch.tril((torch.ones(T,T)))
weight=weight.masked_fill(tril==0),float('-inf')
weight=F.softmax(weights,dim=-1)


v=value(x)
output=weight @ v

## important
## key and query : reprsent what we are interrested in
## x : represnt what we have
## v : represent the vector so that selected after dot product of volleyball


output.shape